1.0 - Imports

In [1]:
# Importando todas as bibliotecas utilizadas

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import warnings


2.0 - Carregamento dos dados

In [6]:
# Carregamento Dataset do site "http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01"

df = pd.read_csv('https://github.com/Xicojr/TechChallenge1/raw/main/Dados/ExpVinho.csv',sep=';')
df.head(10)

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2018,2018.1,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,11,46,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,0,0,26,95,4,21,0,0,0,0
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
3,4,Angola,0,0,0,0,0,0,0,0,...,477,709,345,1065,0,0,0,0,4068,4761
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,Antígua e Barbuda,0,0,0,0,0,0,0,0,...,37,191,219,1549,624,1864,805,2268,419,1866
6,7,Antilhas Holandesas,280,207,4800,3705,3000,1936,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,Argentina,0,0,0,0,0,0,0,0,...,15711,59150,0,0,1015,4176,6,13,480,3232
8,9,Aruba,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Austrália,0,0,0,0,0,0,0,0,...,2055,6902,1161,4682,1013,3413,705,4034,1424,12299


3.0 - EDA

In [11]:

# Melt the DataFrame
df_melted = pd.melt(df, id_vars=['Id', 'País'], var_name='Year_Metric', value_name='Value')

# Split 'Year_Metric' into separate 'Year' and 'Metric' columns
df_melted['Year'] = df_melted['Year_Metric'].apply(lambda x: int(str(x).split('.')[0]))
df_melted['Metric'] = df_melted['Year_Metric'].apply(lambda x: 'Quantity' if '.1' not in str(x) else 'Value_USD')

# Drop the 'Year_Metric' column as it's no longer needed
df_melted.drop('Year_Melted', axis=1, inplace=True, errors='ignore')
df_melted.drop('Year_Metric', axis=1, inplace=True, errors='ignore')

# Pivot the table to have 'Quantity' and 'Value_USD' as separate columns for each year and country
df_pivot = df_melted.pivot_table(index=['Id', 'País', 'Year'], columns='Metric', values='Value', aggfunc='first').reset_index()

# Add the 'País de Origem' column with 'Brazil' as the default value
df_pivot['País de Origem'] = 'Brazil'

# Rename the columns to match the desired output
df_pivot.rename(columns={'País': 'País de Destino', 'Quantity': 'Quantidade em litros exportada', 'Value_USD': 'Valor em US$'}, inplace=True)

# Reorder the columns to match the desired format
df_pivot = df_pivot[['País de Origem', 'País de Destino', 'Year', 'Quantidade em litros exportada', 'Valor em US$']]

# Display the first few rows of the reshaped DataFrame
df_pivot.tail(20)


Metric,País de Origem,País de Destino,Year,Quantidade em litros exportada,Valor em US$
6764,Brazil,Vietnã,2003,0,0
6765,Brazil,Vietnã,2004,0,0
6766,Brazil,Vietnã,2005,0,0
6767,Brazil,Vietnã,2006,2013,2362
6768,Brazil,Vietnã,2007,0,0
6769,Brazil,Vietnã,2008,0,0
6770,Brazil,Vietnã,2009,743,2143
6771,Brazil,Vietnã,2010,8820,9977
6772,Brazil,Vietnã,2011,0,0
6773,Brazil,Vietnã,2012,0,0


In [16]:
# Filter the DataFrame to only include data from the years 2008 to 2019
df_filtered = df_pivot[df_pivot['Year'].between(2008, 2022)]

df_filtered.head()

Metric,País de Origem,País de Destino,Year,Quantidade em litros exportada,Valor em US$
38,Brazil,Afeganistão,2008,0,0
39,Brazil,Afeganistão,2009,0,0
40,Brazil,Afeganistão,2010,0,0
41,Brazil,Afeganistão,2011,0,0
42,Brazil,Afeganistão,2012,0,0


In [26]:
# Group the data by 'País de Origem' and 'País de Destino' and sum the quantities and values
df_grouped = df_filtered.groupby(['País de Origem', 'País de Destino']).agg({
    'Quantidade em litros exportada': 'sum',
    'Valor em US$': 'sum'
}).reset_index()

# Keep only the desired columns
df_final = df_grouped[['País de Origem', 'País de Destino', 'Quantidade em litros exportada', 'Valor em US$']]

# Display the first few rows of the final DataFrame
df_final.head(10)

Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$
0,Brazil,Afeganistão,11,46
1,Brazil,"Alemanha, República Democrática",909051,2546394
2,Brazil,Angola,168868,576189
3,Brazil,Anguilla,0,0
4,Brazil,Antilhas Holandesas,97002,142915
5,Brazil,Antígua e Barbuda,2104,7738
6,Brazil,Argentina,51012,221684
7,Brazil,Aruba,1350,2435
8,Brazil,Austrália,274654,469248
9,Brazil,Bahamas,14101,76628


In [28]:
# Format the numeric columns to have commas as thousands separators
# Sort the DataFrame by the 'Quantidade em litros exportada' column in descending order
df_sorted = df_final.sort_values(by='Quantidade em litros exportada', ascending=False).reset_index(drop=True)
df_sorted['Quantidade em litros exportada'] = df_sorted['Quantidade em litros exportada'].apply(lambda x: '{:,}'.format(x))
df_sorted['Valor em US$'] = df_sorted['Valor em US$'].apply(lambda x: '{:,}'.format(x))

# Display the first few rows of the DataFrame with the formatted numbers
df_sorted.head()

Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$
0,Brazil,Rússia,"39,029,799","25,504,484"
1,Brazil,Paraguai,"29,214,770","38,719,031"
2,Brazil,Estados Unidos,"3,563,355","9,684,567"
3,Brazil,China,"2,509,458","4,746,525"
4,Brazil,Espanha,"1,993,000","3,808,552"


3.1 - Tabela com Top 15 países importadores de vinho brasileiro em litros

In [29]:
# Replace Alemanha, República Democrática => Alemanha
df_sorted['País de Destino'] = df_sorted['País de Destino'].replace('Alemanha, República Democrática', 'Alemanha')

# Display the first few rows of the DataFrame with the formatted numbers
df_sorted.head(15)

Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$
0,Brazil,Rússia,"39,029,799","25,504,484"
1,Brazil,Paraguai,"29,214,770","38,719,031"
2,Brazil,Estados Unidos,"3,563,355","9,684,567"
3,Brazil,China,"2,509,458","4,746,525"
4,Brazil,Espanha,"1,993,000","3,808,552"
5,Brazil,Haiti,"1,791,603","2,327,208"
6,Brazil,Reino Unido,"1,239,551","4,711,464"
7,Brazil,Países Baixos,"1,236,154","3,791,611"
8,Brazil,Japão,"1,181,692","2,377,716"
9,Brazil,Alemanha,"909,051","2,546,394"


3.2 - Data Analysis

In [44]:
# fixes
df_final['País de Destino'] = df_final['País de Destino'].replace('Alemanha, República Democrática', 'Alemanha')
df_final['País de Destino'] = df_final['País de Destino'].replace('Eslovaca, Republica', 'Eslovaca')


# Convert the formatted numbers back to integers for calculations
df_sorted = df_final.sort_values(by='Quantidade em litros exportada', ascending=False).reset_index(drop=True)
df_sorted['Quantidade em litros exportada'] = df_sorted['Quantidade em litros exportada'].apply(lambda x: '{:,}'.format(x))
df_sorted['Valor em US$'] = df_sorted['Valor em US$'].apply(lambda x: '{:,}'.format(x))
df_sorted['Quantidade em litros exportada'] = df_sorted['Quantidade em litros exportada'].str.replace(',', '').astype(int)
df_sorted['Valor em US$'] = df_sorted['Valor em US$'].str.replace(',', '').astype(int)

# 1. Top Export Destinations by Value
df_top_value = df_sorted.sort_values(by='Valor em US$', ascending=False).head()
print("\nTop 5 Exportações por Valor (US$)")
display(df_top_value.head())

# 2. Least Export Destinations by Quantity
df_least_quantity = df_sorted[df_sorted['Quantidade em litros exportada'] > 0].sort_values(by='Quantidade em litros exportada').head()
print("\nTop -5 Exportações por Valor (US$)")
display(df_least_quantity.head())

# 3. Value-to-Quantity Ratio
df_sorted['Value-to-Quantity Ratio'] = df_sorted['Valor em US$'] / df_sorted['Quantidade em litros exportada']
df_sorted['Value-to-Quantity Ratio'] = df_sorted['Value-to-Quantity Ratio'].fillna(0)  # Handle divisions by zero
df_top_ratio = df_sorted.sort_values(by='Value-to-Quantity Ratio', ascending=False).head()
print("\nTop 5 Países com maior valor em relação a litro exportado (US$)")
display(df_top_ratio.head()) ## Contryes with highest value generated per liter exported


# 4. Data Distribution
mean_quantity = df_sorted['Quantidade em litros exportada'].mean()
median_quantity = df_sorted['Quantidade em litros exportada'].median()
std_dev_quantity = df_sorted['Quantidade em litros exportada'].std()

mean_value = df_sorted['Valor em US$'].mean()
median_value = df_sorted['Valor em US$'].median()
std_dev_value = df_sorted['Valor em US$'].std()

print("Média", mean_value)
print("Mediana", median_value)
print("Desvio Padrão", std_dev_value)



Top 5 Exportações por Valor (US$)


Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$
1,Brazil,Paraguai,29214770,38719031
0,Brazil,Rússia,39029799,25504484
2,Brazil,Estados Unidos,3563355,9684567
3,Brazil,China,2509458,4746525
6,Brazil,Reino Unido,1239551,4711464



Top -5 Exportações por Valor (US$)


Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$
116,Brazil,Tuvalu,2,4
115,Brazil,Indonésia,5,6
113,Brazil,Belice,9,29
114,Brazil,Mauritânia,9,85
112,Brazil,Comores,9,25



Top 5 Países com maior valor em relação a litro exportado (US$)


Metric,País de Origem,País de Destino,Quantidade em litros exportada,Valor em US$,Value-to-Quantity Ratio
83,Brazil,Eslovaca,585,16063,27.458120
94,Brazil,Bulgária,122,1610,13.196721
109,Brazil,Bangladesh,12,133,11.083333
98,Brazil,Croácia,55,607,11.036364
85,Brazil,Letônia,387,3723,9.620155


Média 880033.71875
Mediana 17079.5
Desvio Padrão 4203946.039930686


# 4.0 - Conclusão da Análise

# Análise Exploratória dos Dados de Exportação de Vinho
## 1. Principais Destinos de Exportação por Valor
Os países onde é gerado o maior valor a partir das exportações de vinho são:

- Rússia: $25,064,390

- Paraguai: $20,501,133

- Estados Unidos: $8,422,327

- Reino Unido: $4,368,194

- Espanha: $3,808,426

## 2. Menores Destinos de Exportação por Quantidade
Os países que recebem a menor quantidade de vinho (onde a quantidade > 0) são:

- Equador: 2 litros

- Montenegro: 9 litros

- Belize: 9 litros

- Serra Leoa: 18 litros

- Gibraltar: 23 litros


## 3. Razão Valor-por-Quantidade
Países com o maior valor gerado por litro exportado:

- Serra Leoa: $39,83 por litro

- República Eslovaca: $27,46 por litro

- Bulgária: $13,50 por litro

- Barbados: $10,94 por litro

- Letônia: $9,62 por litro

## 4. Distribuição dos Dados
- Quantidade:
  - Média: 535,063 litros

  - Mediana: 1,279 litros

  - Desvio Padrão: 3,653,642 litros

- Valor:
  - Média: $670,069

  - Mediana: $5,092

  - Desvio Padrão: $2,996,849